In [3]:
from transformers import BertModel, PreTrainedModel
from torch import nn
class CustomBERTModel(PreTrainedModel):
      def __init__(self, config, transformer_model_name_or_path, num_feats):
            super(CustomBERTModel, self).__init__(config)
            self.bert = BertModel.from_pretrained(
                  transformer_model_name_or_path,
                  config=config
            )
            ### New layers:
            self.regression_layer = nn.Sequential(
                  nn.Linear(config.hidden_size, 384),
                  nn.LeakyReLU(),
                  nn.Linear(384, 192),
                  nn.LeakyReLU(),
                  nn.Linear(192, 96),
                  nn.LeakyReLU(),
                  nn.Linear(96, 48),
                  nn.LeakyReLU(),
                  nn.Linear(48, 24),
                  nn.LeakyReLU(),
                  nn.Linear(24, 12),
                  nn.LeakyReLU(),
                  nn.Linear(12, num_feats),
                  nn.Sigmoid(),
            )

      def forward(self, **inputs):
            bert_outputs = self.bert(**inputs)
            logits = self.regression_layer(bert_outputs.pooler_output)
            return logits

      def _init_weights(self, module):
            self.bert._init_weights(module)

[2024-01-07 14:55:10,482] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
from transformers import AutoConfig, AutoTokenizer
import torch
base_model = 'bert-base-uncased'
checkpoint = '/home/yuuhanase/FPTU/EXE101/PaperClipAI_EnglishGrading/EnglishGradingModel'
config = AutoConfig.from_pretrained(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)
# model = CustomBERTModel(config=config, transformer_model_name_or_path=base_model, num_feats=6).to("cuda") ## can be gpu
model = CustomBERTModel.from_pretrained(
    checkpoint, 
    config=config, 
    transformer_model_name_or_path=base_model,
    num_feats=6
)

In [10]:
inp_text = "When a problem is a change you have to let it do the best on you no matter what is happening it can change your mind. sometimes you need to wake up and look what is around you because problems are the best way to change what you want to change along time ago. A problem is a change for you because it can make you see different and help you to understand how tings wok. First of all it can make you see different then the others. For example i remember that when i came to the United States i think that nothing was going to change me because i think that nothing was going to change me because everything was different that my country and then i realist that wrong because a problem may change you but sometimes can not change the way it is, but i remember that i was really shy but i think that change a lot because sometimes my problems make me think that there is more thing that i never see in my life but i just need to see it from a different way and dont let nothing happened and ruing the change that i want to make because of just a problem. For example i think that nothing was going to change me and that i dont need to be shy anymore became i need to start seeing everything in a different ways because you can get mad at every one but you need to know what is going to happened after, people may see you different but the only way that you know how to change is to do the best and don't let nothing or not body to change nothing about you. The way you want to change not one have that and can't do nothing about it because is your choice and your problems and you can decide what to do with it. second of all can help you to understand how things work. For instance my mom have a lot of problems but she have faith when she is around people, my mom is scare of high and i'm not scare of high i did not understand why my mos is scare of high and in not scare of high and every time i see my mom in a airplane it make me laugh because she is scare and is funny, but i see it from a different way and i like the high but also she have to understand that hoe things work in other people because it can no be the same as you. For example i think that my mom and me are different because we are and i have to understand that she does not like high and i need to understand that. to help someone to understand how things work you need to start to see how things work in that persons life. A problem is a change for you and can make you a different and help you to understand. Everyone has a different opinion and a different was to understand then others. everyone can see the different opinion and what other people think."
#2.5,2.5,3.0,2.0,2.0,2.5
tokenized_input = tokenizer(inp_text, return_tensors='pt', truncation=True)#.to(torch.device("cuda"))
output = model(**tokenized_input)[0]
feats = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
result = {}
for i in range(6):
    result[feats[i]] = output[i].item()*5.0
result

{'cohesion': 2.383091002702713,
 'syntax': 2.0666779577732086,
 'vocabulary': 2.884359359741211,
 'phraseology': 2.844240665435791,
 'grammar': 2.9789209365844727,
 'conventions': 2.2683973610401154}

In [10]:
from transformers import Trainer
from torch import nn
class CustomTrainer(Trainer):
    def __int__(self, *args, **kwargs):
        super().__int__(*args, **kwargs)
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # forward pass
        logits = model(**inputs)
        loss_fct = nn.MSELoss()
        loss = loss_fct(logits, labels)
        return (loss, logits) if return_outputs else loss

In [ ]:
trainer